In [1]:
import utils

In [2]:
labels="'earthquake', 'quake', 'temblor'"
time=True
participants=False
location=True
strictLocations=True
pageRankFilter=True
pr_limit=500.0

In [3]:
query=utils.get_me_a_query(labels, time, location, participants, strictLocations, pageRankFilter, pr_limit)
print(query)


	PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
	PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
	PREFIX dbpedia: <http://dbpedia.org/resource/>
	PREFIX owltime: <http://www.w3.org/TR/owl-time#>
	PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	PREFIX dbo: <http://dbpedia.org/ontology/>
	PREFIX vrank: <http://purl.org/voc/vrank#>
	SELECT ?event ?location ?time ?participant WHERE {
    
		?event a sem:Event .
		{ ?event rdfs:label ?label .
		FILTER (?label IN ('earthquake', 'quake', 'temblor')) } 
		UNION
		{ ?event sem:hasActor ?nonentity .
		?nonentity skos:relatedMatch dbpedia:Earthquake } . 
		?event sem:hasTime ?tmx . ?tmx owltime:inDateTime ?time .  
		?event sem:hasPlace ?location .  
		?location a ?x . ?x rdfs:subClassOf* dbo:Place . 
		FILTER (!REGEX(STR(?x), 'http://dbpedia.org/ontology/Country')) .
		FILTER NOT EXISTS { ?x rdfs:subClassOf* dbo:Country  } .   
		?location vrank:hasRank/vrank:rankValue ?pagerank .
		FILTER (xsd:float(?pagerank)<=500.000000) .


In [4]:
res=utils.get_sparql_results(query)
len(res)

84

In [36]:
from collections import Counter
from collections import defaultdict
import operator
import json
from Levenshtein import ratio

In [21]:
res[0]

{'event': {'type': 'uri',
  'value': 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/d7ddc614-701e-4700-917a-0efc25aa3459.in.naf#ev59'},
 'location': {'type': 'uri',
  'value': 'http://dbpedia.org/resource/Banda_Aceh'},
 'time': {'type': 'uri',
  'value': 'http://www.newsreader-project.eu/time/2004'}}

In [27]:
results = defaultdict(set)
for item in res:
    key = (item['location']['value'], item['time']['value'])
    results[key].add(item['event']['value'])

In [23]:
apath_signal_media_json = 'SignalMedia/signalmedia-1m.jsonl'

In [42]:
def retrieve_ids(path_signal_media_json, ids):
    result_jsons = dict()
    
    with open(path_signal_media_json) as infile:
        for counter, line in enumerate(infile, 1):
            a_json = json.loads(line)
            if a_json['id'] in ids:
                result_jsons[a_json['id']] = a_json
            
            if set(result_jsons.keys()) == ids:
                break
    
    return result_jsons

In [43]:
def find_duplicates(ids, results_json, threshold=0.8):
    possible_duplicates = set()
    
    for an_id1 in ids:
        for an_id2 in ids:
            if an_id1 != an_id2:
                distance = ratio(result_jsons[an_id1]['content'],
                                 result_jsons[an_id2]['content'])
                if distance > threshold:
                    possible_duplicates.add(an_id1)
                    possible_duplicates.add(an_id2)
    return possible_duplicates
    

In [28]:
needed_ids = {ev_id.split('/')[-1].split('.')[0]
              for value in results.values()
              for ev_id in value}
result_jsons = retrieve_ids(apath_signal_media_json,
                            needed_ids)

In [71]:
done = {('http://dbpedia.org/resource/Cushing,_Oklahoma', 'http://www.newsreader-project.eu/time/20150918'),
       ('http://dbpedia.org/resource/Iquique', 'http://www.newsreader-project.eu/time/201404'),    
       ('http://dbpedia.org/resource/Queensland', 'http://www.newsreader-project.eu/time/20150801'),
       ('http://dbpedia.org/resource/Queensland', 'http://www.newsreader-project.eu/time/20150731'),
       ('http://dbpedia.org/resource/Washington_(state)', 'http://www.newsreader-project.eu/time/20150901'),
       ('http://dbpedia.org/resource/Oklahoma', 'http://www.newsreader-project.eu/time/201508'),
        ('http://dbpedia.org/resource/Sinaloa', 'http://www.newsreader-project.eu/time/20150913'),
('http://dbpedia.org/resource/Papua_(province)', 'http://www.newsreader-project.eu/time/20150925'),
        ('http://dbpedia.org/resource/Grand_Coulee_Dam', 'http://www.newsreader-project.eu/time/20150901'),
        ('http://dbpedia.org/resource/Baja_California', 'http://www.newsreader-project.eu/time/20150913'),
        ('http://dbpedia.org/resource/Illapel', 'http://www.newsreader-project.eu/time/20150917'),
        ('http://dbpedia.org/resource/Fraser_Island', 'http://www.newsreader-project.eu/time/20150801'),
        ('http://dbpedia.org/resource/Fraser_Island', 'http://www.newsreader-project.eu/time/20150731')

}

In [72]:
counter = 0
for key, value in results.items():
    if all([len(value) >= 2,
            key not in done]):
        ids = {ev_id.split('/')[-1].split('.')[0]
               for ev_id in value}
        print()
        print(key)
        duplicates = find_duplicates(ids, result_jsons, threshold=0.8)
        print()
        print('possible duplicates')
        print(duplicates)
        print()
        for an_id in ids:
            print()
            print(result_jsons[an_id])
        break
        
